In [101]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import random
import pickle
import os
import argparse

parser = argparse.ArgumentParser(description='This is a demon')
parser.add_argument('-batch_size', type=str, required=True, help='provode batch size')

args= parser.parse_args()
print(f'batch_size: {args.batch_size}')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
#todo autotuning
block_size = 8
batch_size = 4
max_iters = 2000
learning_rate = 3e-4
eval_iters=500
n_embd = 400
n_head = 4
n_layer=4
dropout=0.2

usage: ipykernel_launcher.py [-h] -batch_size BATCH_SIZE
ipykernel_launcher.py: error: the following arguments are required: -batch_size


SystemExit: 2

C:\Users\patron\Documents\research\ffc\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [81]:
chars = ""  
with open('vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\x00', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x7f', '\x80', '\x81', '\x82', '\x83', '\x84', '\x86', '\x87', '\x88', '\x89', '\x8a', '\x8b', '\x8d', '\x8e', '\x8f', '\x90', '\x91', '\x92', '\x93', '\x94', '\x95', '\x96', '\x97', '\x98', '\x99', '\x9a', '\x9c', '\x9d', '\x9e', '\x9f', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', '

In [82]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

#data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [83]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
def get_random_chunk(split):
    filename ="output_train.txt" if split == 'train' else "output_val.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_pos=random.randint(0, (file_size) - block_size)
            mm.seek(start_pos)
            block=mm.read(block_size*batch_size-1)
            decoded_block=block.decode('utf-8', errors='ignore').replace('\r', '')
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data
        
def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')

In [84]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
        

In [85]:
class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key=nn.Linear(n_embd, head_size, bias=False)
        self.query=nn.Linear(n_embd, head_size, bias=False)
        self.value=nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size,block_size)))
        self.dropout=nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k=self.key(x)
        q=self.query(x)
        wei=q @ k.transpose(-2, -1)*k.shape[-1]**-0.5
        wei=wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei=self.dropout(wei)
        v=self.value(x)
        out = wei @ v
        return out

In [86]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(head_size*num_heads, n_embd)
        self.dropout=nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out=self.dropout(self.proj(out))
        return out

In [87]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size=n_embd//n_head
        self.sa=MultiHeadAttention(n_head, head_size)
        self.ffwd=FeedForward(n_embd)
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd) 

    def forward(self, x):
        y = self.sa(x)
        x=self.ln1(x+y)
        y=self.ffwd(x)
        x=self.ln2(x+y)
        return x
        

In [92]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.positional_embedding = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
    def forward(self, index, targets=None):
        B,T= index.shape
        token_emd = self.token_embedding_table(index)
        pos_emd = self.positional_embedding(torch.arange(T, device = device))
        x = token_emd + pos_emd
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):

        for _ in range(max_new_tokens):
            index_cond = index[:, -block_size:]
            logits, loss = self.forward(index_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples =1)
            index = torch.cat((index, index_next), dim=1)

        #print(index)
        return index

model = GPTLanguageModel(vocab_size)
#if os.path.exists('./model-01.pkl'):
with open('model-01.pkl', 'rb') as f:
    model = pickle.load(f)

m = model.to(device)

#context = torch.zeros((1,1), dtype=torch.long, device = device)
#generated_chars = decode(m.generate(context, 500)[0].tolist())
#print(generated_chars)

In [93]:
@torch.no_grad()
def estimate_loss():
    out ={}
    model.eval()
    for tt in ['train', 'test']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(tt)
            logits, loss=model(X,Y)
            losses[k] = loss.item()
        out[tt] = losses.mean()
    model.train()
    return out


In [97]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter%eval_iters==0:
        losses=estimate_loss()
        print(f'step:{iter}, {losses}')
    x, y = get_batch('train')
    logits, loss = model.forward(x,y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
#print(loss)
#print(loss.item())

with open('model-01.pkl', "wb") as f:
    pickle.dump(model, f)
print('model saved')

step:0, {'train': tensor(2.4710), 'test': tensor(2.4991)}
step:500, {'train': tensor(2.3150), 'test': tensor(2.3161)}
step:1000, {'train': tensor(2.4161), 'test': tensor(2.3185)}
step:1500, {'train': tensor(2.2854), 'test': tensor(2.3025)}
model saved


In [95]:
#prompt = 'Hello! Can you see me?'
#context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
#generated_chars = decode(m.generate(context.unsqueeze(0), max_new_tokens=100)[0].tolist())
#print(generated_chars)

context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)

                               ”iond bS7,0000000000                                                                                                                                                         apa ulnth2
UPA0047950 and C

N
Th


A
U6P86,1000  arighatieanionti2

NO205,

B
“NBꦯB
iliaf)
07TA000
J)RN
ANc ama
SA
"N1'65't S’-90 A, t rele-.

S62




“LED905300ath outithatin AA.. 2an,eicistimS’595100080T

2ldSB8 E3."L S
S14015D
6l
S%
5ea y1B60506T00                                             
